# Recomendador de planes

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

## Carga de datos y librerías

In [31]:
# Cargar todas las librerías
import pandas as pd
from sklearn.model_selection import train_test_split #Librería para poder dividir los dataframes
from sklearn.tree import DecisionTreeClassifier #Librería para el modelado por árbol de decisión
from sklearn.ensemble import RandomForestClassifier #Librería para el modelado por bosque aleatorio
from sklearn.linear_model import LogisticRegression #Librería para el modelado por regresión logística
from sklearn.metrics import accuracy_score #Librería para el calculo de la exactitud de los modelos

### Descripción de los datos

- сalls — número de llamadas
- minutes — duración total de la llamada en minutos
- messages — número de mensajes de texto
- mb_used — Tráfico de Internet utilizado en MB
- is_ultra — plan para el mes actual (Ultra - 1, Smart - 0)

In [2]:
# Carga los archivos de datos 
df = pd.read_csv('users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Análisis de datos

In [4]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [5]:
df.duplicated().sum()

0

Comprobamos que el dataset no presenta duplicados ni valores ausentes, por lo tanto, no se realizarán mayores modificaciones en el dataset.

## Segmentación de datos

In [17]:
features = train.drop('is_ultra', axis=1)
target = train['is_ultra']

Dividiremos los datos en 3 secciones: entrenamiento, validación y prueba. Se dividirán los siguientes porcentajes:

- 60% entrenamiento
- 20% validación
- 20% prueba

In [6]:
train_val, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train_val, test_size=0.25, random_state=42)

In [7]:
print('Procentaje del dataframe de entrenamiento:',train.shape[0]/df.shape[0]*100)
print('Porcentaje del dataframe de validación:',val.shape[0]/df.shape[0]*100)
print('Porcentaje del dataframe de prueba:',test.shape[0]/df.shape[0]*100)

Procentaje del dataframe de entrenamiento: 59.98755444928439
Porcentaje del dataframe de validación: 20.00622277535781
Porcentaje del dataframe de prueba: 20.00622277535781


Comprobamos que los dataframes contienen la cantidad de filas adecuadas conforme a los porcentajes establecidos.

Generamos las secciones features y target para cada dataframe. Para features usaremos todos las columnas menos is_ultra y para target usaremos is_ultra.

In [12]:
train_features = train.drop('is_ultra', axis=1)
train_target = train['is_ultra']
val_features = val.drop('is_ultra', axis=1)
val_target = val['is_ultra']
test_features = test.drop('is_ultra', axis=1)
test_target = test['is_ultra']

## Modelado de datos

En esta sección calcularemos y compararemos la exactitud de 3 módelos (árbol de decisión, bosque aleatorio y regresión logística) para determinar el que tenga una exactitud más cercana, igual o superior al objetivo que tenemos que es de 0.75

### Árbol de decisión

In [20]:
#Entrenamos el modelo con los datos de entrenamiento y calculamos la exactitud
model = DecisionTreeClassifier(random_state=12345)
model.fit(features, target)
train_predictions = model.predict(features)
print('Exactitud Training set:', accuracy_score(target, train_predictions))

test_predictions = model.predict(test_features)
print('Exactitud Test set:', accuracy_score(test_target, test_predictions))

val_predictions = model.predict(val_features)
print('Exactitud Validation set:', accuracy_score(val_target, val_predictions))

Exactitud Training set: 1.0
Exactitud Test set: 0.7340590979782271
Exactitud Validation set: 0.7231726283048211


Sin ajustar los hiperparámetros, observamos que el modelo no cuenta con la exactitud requerida, así que procederemos a ajustar para verificar si podemos aumentar la exactitud.

In [23]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(train_features, train_target)
    predictions_valid = model.predict(val_features)
    print('max_depth =', depth, ': ', end='')
    print(accuracy_score(val_target, predictions_valid))

max_depth = 1 : 0.7418351477449455
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7744945567651633
max_depth = 4 : 0.7807153965785381
max_depth = 5 : 0.7713841368584758


In [27]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(train_features, train_target)
    predictions_test = model.predict(test_features)
    print('max_depth =', depth, ': ', end='')
    print(accuracy_score(test_target, predictions_test))

max_depth = 1 : 0.7698289269051322
max_depth = 2 : 0.7900466562986003
max_depth = 3 : 0.7776049766718507
max_depth = 4 : 0.7900466562986003
max_depth = 5 : 0.8040435458786936


Al ajustar la profundidad máxima observamos que al utilizar una profundidad de 4 el modelo alcanza una exactitud de 0.781 para el conjunto de validación y 0.790 para el conjunto de prueba, la cual es superior a la esperada, pero seguiremos probando otros modelos para verificar si logramos conseguir una exactitud mayor.

### Bosque aleatorio

Para el modelo de bosque aleatorio variaremos el número de árboles para encontrar el modelo con mayor exactitud.

In [28]:
best_score = 0
best_est = 0
for est in range(1, 11): # selecciona el rango del hiperparámetro
    model = RandomForestClassifier(random_state=54321, n_estimators=est) # configura el número de árboles
    model.fit(train_features, train_target) # entrena el modelo en el conjunto de entrenamiento
    score = model.score(val_features, val_target) # calcula la puntuación de accuracy en el conjunto de validación
    if score > best_score:
        best_score = score # guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de accuracy

print("Exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

Exactitud del mejor modelo en el conjunto de validación (n_estimators = 8): 0.7900466562986003


In [29]:
best_score = 0
best_est = 0
for est in range(1, 11): # selecciona el rango del hiperparámetro
    model = RandomForestClassifier(random_state=54321, n_estimators=est) # configura el número de árboles
    model.fit(train_features, train_target) # entrena el modelo en el conjunto de entrenamiento
    score = model.score(test_features, test_target) # calcula la puntuación de accuracy en el conjunto de validación
    if score > best_score:
        best_score = score # guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de accuracy

print("Exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

Exactitud del mejor modelo en el conjunto de validación (n_estimators = 8): 0.7978227060653188


Al ajustar el número de árboles observamos que al utilizar un valor de 8 el modelo alcanza una exactitud de 0.790 para el conjunto de validación y 0.798 para el conjunto de prueba, la cual es superior a la esperada, pero seguiremos probando otros modelos para verificar si logramos conseguir una exactitud mayor.

### Regresión logística

In [33]:
model = LogisticRegression(random_state=54321, solver='liblinear')  # inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model.fit(train_features, train_target) # entrena el modelo en el conjunto de entrenamiento
score_train = model.score(train_features, train_target) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_val = model.score(val_features, val_target) # calcula la puntuación de accuracy en el conjunto de validación
score_test = model.score(test_features, test_target) # calcula la puntuación de accuracy en el conjunto de prueba

print("Exactitud del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo de regresión logística en el conjunto de validación:", score_val)
print("Exactitud del modelo de regresión logística en el conjunto de prueba:", score_test)

Exactitud del modelo de regresión logística en el conjunto de entrenamiento: 0.703838174273859
Exactitud del modelo de regresión logística en el conjunto de validación: 0.7216174183514774
Exactitud del modelo de regresión logística en el conjunto de prueba: 0.702954898911353


Para este dataset, el modelo de regresión logística parecer no ser el más adecuado ya que para el conjunto de validación se obtuvo una exactitud de 0.721 y para el conjunto de prueba fue de 0.703, lo cual, es inferior a lo obtenido por los otros modelos e incluso inferior al valor esperado de exactitud.

## Conclusiones

Después de realizar varias pruebas con los diferentes modelos y ajustar los hiperparámetros se observa que los modelos que mejores resultados arrojaron fue árbol de decisión y bosque aleatorio, ambos teniendo una exactitud superior a la esperada.

Cualquiera de estos 2 modelos podrían ser utilizados para seleccionar el mejor plan a recomendar para los usuarios con planes heredados. Si quisiéramos elegir el que mayor exactitud presentó deberíamos seleccionar el modelo bosque aleatorio con una cantidad de árboles igual a 8, pero quisiera concluir el proyecto dando un poco más de información respecto al mejor modelo.

Ya que si priorizamos la velocidad del modelo, podríamos sacrificar un poco la exactitud y seleccionar el modelo de árbol de decisión y tener un excelente desempeño, personalmente sería la opción que elegiría, pero como se mencionó en el párrafo anterior, si queremos la mayor exactitud, que podría reflejar mayores ganancias, el modelo bosque aleatorio debería ser elegido.